In [3]:
# 한국투자증권 API 를 쉽게 사용하기 위한 라이브러리 입니다.
# !pip install mojito2

In [1]:
import mojito
from config import APP_KEY, SECRET_KEY, ACCOUNT

broker = mojito.KoreaInvestment(
    api_key=APP_KEY,
    api_secret=SECRET_KEY,
    acc_no=ACCOUNT,
    mock=True
)

print(broker)

50090274
